In [1]:
import numpy as np
import math
import torch
from torch import nn

import scipy.stats as stats
import matplotlib
import matplotlib.pyplot as plt

from tempfile import TemporaryDirectory

from Models import BigGenerator
from Tools import KL, batchKL, sw, FunSW, FunKL

from tqdm import trange

from Experiments import get_setup

import itertools

from pandas import DataFrame
import pandas as pd

print(torch.__version__)


1.4.0


In [2]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda', index=0)

In [3]:
lat_dim=5

In [4]:
models={'foong':{'HMC': None,
                 'GeNNeVI':None,
                 'FuNNeVI':None},
        'foong_sparse':{'HMC': None,
                 'GeNNeVI':None,
                 'FuNNeVI':None},
        'boston':{'HMC': None,
                 'GeNNeVI':None,
                 'FuNNeVI':None},
        'concrete':{'HMC':None,
                 'GeNNeVI':None,
                 'FuNNeVI':None},
        'energy':{'HMC':None,
                 'GeNNeVI':None,
                 'FuNNeVI':None},
       'powerplant':{'HMC': None,
                 'GeNNeVI':None,
                 'FuNNeVI':None},
       'wine':  {'HMC': None,
                 'GeNNeVI':None,
                 'FuNNeVI':None},
       'yacht': {'HMC': None,
                 'GeNNeVI':None,
                 'FuNNeVI':None}}
pd.DataFrame(models)

,foong,foong_sparse,boston,concrete,energy,powerplant,wine,yacht
HMC,None,None,None,None,None,None,None,None
GeNNeVI,None,None,None,None,None,None,None,None
FuNNeVI,None,None,None,None,None,None,None,None


In [5]:
datasets=[d for d,i in models.items()]
datasets

['foong',
 'foong_sparse',
 'boston',
 'concrete',
 'energy',
 'powerplant',
 'wine',
 'yacht']

In [6]:
metrics=['KL(G,HMC)','KL(F,HMC)','KL(HMC,G)','KL(HMC,F)','SW(HMC,G)','SW(HMC,F)','KL(G,G)', 'KL(F,F)','SW(G,G)', 'SW(F,F)']
results_f={m:{d:None for d in datasets} for m in metrics}


In [7]:
pd.DataFrame(results_f)

,"KL(G,HMC)","KL(F,HMC)","KL(HMC,G)","KL(HMC,F)","SW(HMC,G)","SW(HMC,F)","KL(G,G)","KL(F,F)","SW(G,G)","SW(F,F)"
foong,None,None,None,None,None,None,None,None,None,None
foong_sparse,None,None,None,None,None,None,None,None,None,None
boston,None,None,None,None,None,None,None,None,None,None
concrete,None,None,None,None,None,None,None,None,None,None
energy,None,None,None,None,None,None,None,None,None,None
powerplant,None,None,None,None,None,None,None,None,None,None
wine,None,None,None,None,None,None,None,None,None,None
yacht,None,None,None,None,None,None,None,None,None,None


In [8]:
models['boston']['HMC']=[torch.load('Results/Boston/Boston_fda8c422c20c451ab8b48f96218d40e5.pt')]
models['concrete']['HMC']=[torch.load('Results/Concrete/Concrete_4b702e1dd2004fd78617809c301d5674.pt')]
models['energy']['HMC']=[torch.load('Results/Energy/Energy_ce3dca708d9f4bf78ba6b91da908bb82.pt')]
models['foong']['HMC']=[torch.load('Results/Foong/Foong_4785e1739f584a4eae7e2c0a72bf1b14.pt')]
models['powerplant']['HMC']=[torch.load('Results/Powerplant/CCPowerPlant_d9c8a27f896b4efb931a460d907f16c8.pt')]
models['yacht']['HMC']=[torch.load('Results/Yacht/Yacht_73f81f1e9d6e43489bbe9aa4a141cd96.pt')]
models['wine']['HMC']=[torch.load('Results/WineQuality/WineQuality_6480b7067eb441d699a2c76623c3fa25.pt')]
models['foong_sparse']['HMC']=[torch.load('Results/Foong sparse/Foong_sparse_6a1ce9534fa94030a63fcd76913535b9.pt')]


In [9]:
ml_path='mlruns/'

In [10]:
models['boston']['FuNNeVI']=torch.load(ml_path+'1/1d4df1879651452d81686d31ca55e6ea/artifacts/models.pt') #
models['boston']['GeNNeVI']=torch.load(ml_path+'2/d65577d20651444f8380a15ad0e5973f/artifacts/models.pt')

models['yacht']['FuNNeVI']=torch.load(ml_path+'3/f5aabd25e85a4cc6881826fe44fe92d9/artifacts/models.pt')
models['yacht']['GeNNeVI']=torch.load(ml_path+'4/03b949f1880644d79e2b61b4002832e9/artifacts/models.pt')

models['concrete']['FuNNeVI']=torch.load(ml_path+'5/e84add6582b2468ba2043c8e86da611a/artifacts/models.pt')
models['concrete']['GeNNeVI']=torch.load(ml_path+'6/31ad0b13ac8d45de9f919593c46297c2/artifacts/models.pt')

models['energy']['FuNNeVI']=torch.load(ml_path+'7/7cab0591944c4f8da67920e59f3d5021/artifacts/models.pt')
models['energy']['GeNNeVI']=torch.load(ml_path+'8/9c44b9daa6d44728ae5169eae8873f6f/artifacts/models.pt')


models['wine']['FuNNeVI']=torch.load(ml_path+'9/0046c5209a784b85b9885b9c44262344/artifacts/models.pt')
models['wine']['GeNNeVI']=torch.load(ml_path+'10/f4c20705739f47c8ab06257e6478cbff/artifacts/models.pt')

models['powerplant']['FuNNeVI']=torch.load(ml_path+'11/114b9f4a84bd4df9954c2d811f4fae80/artifacts/models.pt')
models['powerplant']['GeNNeVI']=torch.load(ml_path+'12/e317f234c62c4125a46a3955f6e85312/artifacts/models.pt')

models['foong']['FuNNeVI']=torch.load(ml_path+'13/fa49f150f1c3498886733d3954065005/artifacts/models.pt')
models['foong']['GeNNeVI']=torch.load(ml_path+'14/50337dbe98374c858069318fa09f2804/artifacts/models.pt')

#models['foong_mixed']['FuNNeVI']=torch.load(ml_path+'17/07025622b9594b62a8bdf231d1c7957c/artifacts/models.pt')
#models['foong_mixed']['GeNNeVI']=torch.load(ml_path+'18/0a0c7376df1541ada31a795d072bf738/artifacts/models.pt')

models['foong_sparse']['FuNNeVI']=torch.load(ml_path+'17/cfab6105f7744c88b5ab2d8edd6791da/artifacts/models.pt')
models['foong_sparse']['GeNNeVI']=torch.load(ml_path+'18/6b6a09c2dec84203a274d07a162b4ac6/artifacts/models.pt')


In [11]:




torch.save(models, 'Results/models_es.pt')



In [13]:
metric='KL(G,HMC)'
for dataset in datasets:
    method='GeNNeVI'

    setup_ = get_setup(dataset)
    setup=setup_.Setup(device) 
    G=BigGenerator(lat_dim, setup.param_count, device).to(device)
    models_pairs=list(itertools.combinations(models[dataset][method].items(),2))
    def projection(t,s,m):
        return setup.projection(t,s,m,ratio_ood=1)
    
    KLs=[]
    for i,m in models[dataset][method].items():
        t=models[dataset]['HMC'][0].to(device)
        G.load_state_dict(m)
        s=G(t.shape[0]).detach()
        K=FunKL(s,t,projection,device)
        print(K)
        KLs.append(K.item())
    print(KLs)
    results_f[metric][dataset]=(np.mean(KLs).round(decimals=3), np.std(KLs).round(decimals=3))


tensor(106.4762)
tensor(157.7225)
tensor(86.8962)
[106.4761734008789, 157.7224578857422, 86.896240234375]
tensor(51.5948)
tensor(66.3793)
tensor(73.3724)
[51.59479904174805, 66.37930297851562, 73.37239074707031]
tensor(66.4192)
tensor(68.6656)
tensor(64.6251)
[66.4192123413086, 68.66558074951172, 64.62510681152344]
tensor(67.1267)
tensor(65.2505)
tensor(62.2590)
[67.12673950195312, 65.25054168701172, 62.25904846191406]
tensor(35.9414)
tensor(48.7314)
tensor(39.3213)
[35.941410064697266, 48.731353759765625, 39.321346282958984]
tensor(88.2448)
tensor(88.1542)
tensor(93.4262)
[88.24475860595703, 88.15419006347656, 93.42616271972656]
tensor(89.2183)
tensor(82.6138)
tensor(87.7088)
[89.21826171875, 82.61384582519531, 87.7088394165039]
tensor(99.6327)
tensor(102.2686)
tensor(101.8882)
[99.63269805908203, 102.26862335205078, 101.88818359375]


In [14]:
metric='KL(F,HMC)'
for dataset in datasets:
    method='FuNNeVI'

    setup_ = get_setup(dataset)
    setup=setup_.Setup(device) 
    G=BigGenerator(lat_dim, setup.param_count, device).to(device)
    models_pairs=list(itertools.combinations(models[dataset][method].items(),2))

    def projection(t,s,m):
        return setup.projection(t,s,m,ratio_ood=1)
    
    KLs=[]
    for i,m in models[dataset][method].items():
        t=models[dataset]['HMC'][0].to(device)
        G.load_state_dict(m)
        s=G(t.shape[0]).detach()
        K=FunKL(s,t,projection,device)
        print(dataset+': '+str(K.item()))
        KLs.append(K.item())
    print(KLs)
    results_f[metric][dataset]=(np.mean(KLs).round(decimals=3), np.std(KLs).round(decimals=3))


foong: 83.1876449584961
foong: 83.80779266357422
foong: 74.31021118164062
[83.1876449584961, 83.80779266357422, 74.31021118164062]
foong_sparse: 91.30570983886719
foong_sparse: 73.54718017578125
foong_sparse: 78.49858856201172
[91.30570983886719, 73.54718017578125, 78.49858856201172]
boston: 10.528000831604004
boston: 9.31811809539795
boston: 12.089581489562988
[10.528000831604004, 9.31811809539795, 12.089581489562988]
concrete: 12.982897758483887
concrete: 11.052529335021973
concrete: 12.015048027038574
[12.982897758483887, 11.052529335021973, 12.015048027038574]
energy: 13.347862243652344
energy: 13.061806678771973
energy: 14.362741470336914
[13.347862243652344, 13.061806678771973, 14.362741470336914]
powerplant: 18.360807418823242
powerplant: 18.023799896240234
powerplant: 17.93297576904297
[18.360807418823242, 18.023799896240234, 17.93297576904297]
wine: 85.24774169921875
wine: 93.09149169921875
wine: 72.65798950195312
[85.24774169921875, 93.09149169921875, 72.65798950195312]
yacht

In [15]:
metric='KL(HMC,G)'
for dataset in datasets:
    method='GeNNeVI'

    setup_ = get_setup(dataset)
    setup=setup_.Setup(device) 
    G=BigGenerator(lat_dim, setup.param_count, device).to(device)
    models_pairs=list(itertools.combinations(models[dataset][method].items(),2))

    def projection(t,s,m):
        return setup.projection(t,s,m,ratio_ood=1)
    
    KLs=[]
    for i,m in models[dataset][method].items():
        t=models[dataset]['HMC'][0].to(device)
        G.load_state_dict(m)
        s=G(t.shape[0]).detach()
        K=FunKL(t,s,projection,device)
        if dataset == 'powerplant' or dataset == 'wine':
                K=FunKL(t,s,projection,device,k=2)

        print(dataset+': '+str(K.item()))
        KLs.append(K.item())
    print(KLs)
    results_f[metric][dataset]=(np.mean(KLs).round(decimals=3), np.std(KLs).round(decimals=3))


foong: 98.75585174560547
foong: 127.39442443847656
foong: 87.76165771484375
[98.75585174560547, 127.39442443847656, 87.76165771484375]
foong_sparse: 32.22868347167969
foong_sparse: 43.512203216552734
foong_sparse: 46.35527420043945
[32.22868347167969, 43.512203216552734, 46.35527420043945]
boston: 9.393126487731934
boston: 9.139997482299805
boston: 9.816054344177246
[9.393126487731934, 9.139997482299805, 9.816054344177246]
concrete: 6.5109710693359375
concrete: 6.549600124359131
concrete: 6.228888034820557
[6.5109710693359375, 6.549600124359131, 6.228888034820557]
energy: 12.567667007446289
energy: 29.58623504638672
energy: 5.297884464263916
[12.567667007446289, 29.58623504638672, 5.297884464263916]
powerplant: 8.061683654785156
powerplant: 8.356324195861816
powerplant: 7.439362049102783
[8.061683654785156, 8.356324195861816, 7.439362049102783]
wine: 23.909116744995117
wine: 20.939348220825195
wine: 22.533123016357422
[23.909116744995117, 20.939348220825195, 22.533123016357422]
yacht: 

In [16]:
metric='KL(HMC,F)'
for dataset in datasets:
    method='FuNNeVI'

    setup_ = get_setup(dataset)
    setup=setup_.Setup(device) 
    G=BigGenerator(lat_dim, setup.param_count, device).to(device)
    models_pairs=list(itertools.combinations(models[dataset][method].items(),2))

    def projection(t,s,m):
        return setup.projection(t,s,m,ratio_ood=1)

    
    KLs=[]
    for i,m in models[dataset][method].items():
        t=models[dataset]['HMC'][0].to(device)
        G.load_state_dict(m)
        s=G(t.shape[0]).detach()
        K=FunKL(t,s,projection,device)
        if dataset == 'powerplant' or dataset == 'wine':
            K=FunKL(t,s,projection,device,k=2)

        print(dataset+': '+str(K.item()))
        KLs.append(K.item())
    print(KLs)
    results_f[metric][dataset]=(np.mean(KLs).round(decimals=3), np.std(KLs).round(decimals=3))


foong: 63.53779220581055
foong: 46.85319519042969
foong: 111.69561767578125
[63.53779220581055, 46.85319519042969, 111.69561767578125]
foong_sparse: 62.30061340332031
foong_sparse: 51.93782043457031
foong_sparse: 52.59099578857422
[62.30061340332031, 51.93782043457031, 52.59099578857422]
boston: 51.6982421875
boston: 54.049835205078125
boston: 51.62834930419922
[51.6982421875, 54.049835205078125, 51.62834930419922]
concrete: 45.70640563964844
concrete: 47.406394958496094
concrete: 44.85776901245117
[45.70640563964844, 47.406394958496094, 44.85776901245117]
energy: 49.135986328125
energy: 49.41371154785156
energy: 47.495670318603516
[49.135986328125, 49.41371154785156, 47.495670318603516]
powerplant: 68.58905029296875
powerplant: 71.43518829345703
powerplant: 70.06594848632812
[68.58905029296875, 71.43518829345703, 70.06594848632812]
wine: 31.560338973999023
wine: 32.50810623168945
wine: 41.327510833740234
[31.560338973999023, 32.50810623168945, 41.327510833740234]
yacht: 52.05242156982

In [17]:
metric='SW(HMC,G)'
for dataset in datasets:
    method='GeNNeVI'

    setup_ = get_setup(dataset)
    setup=setup_.Setup(device) 
    G=BigGenerator(lat_dim, setup.param_count, device).to(device)

    def projection(t,s,m):
        return setup.projection(t,s,m,ratio_ood=1)
    
    KLs=[]
    for i,m in models[dataset][method].items():
        t=models[dataset]['HMC'][0].to(device)
        G.load_state_dict(m)
        s=G(t.shape[0]).detach()
        K=FunSW(t,s, projection, device)
        print(dataset+': '+str(K.item()))
        KLs.append(K.item())
    print(KLs)
    results_f[metric][dataset]=(np.mean(KLs).round(decimals=3), np.std(KLs).round(decimals=3))



foong: 0.06574838608503342
foong: 0.09694572538137436
foong: 0.042356401681900024
[0.06574838608503342, 0.09694572538137436, 0.042356401681900024]
foong_sparse: 0.015708865597844124
foong_sparse: 0.014754125848412514
foong_sparse: 0.02068604901432991
[0.015708865597844124, 0.014754125848412514, 0.02068604901432991]
boston: 0.42847660183906555
boston: 0.419473260641098
boston: 0.4271101653575897
[0.42847660183906555, 0.419473260641098, 0.4271101653575897]
concrete: 0.3159531056880951
concrete: 0.3181535303592682
concrete: 0.31106922030448914
[0.3159531056880951, 0.3181535303592682, 0.31106922030448914]
energy: 0.34261608123779297
energy: 0.5544866919517517
energy: 0.23954543471336365
[0.34261608123779297, 0.5544866919517517, 0.23954543471336365]
powerplant: 0.10419965535402298
powerplant: 0.1051192581653595
powerplant: 0.10481424629688263
[0.10419965535402298, 0.1051192581653595, 0.10481424629688263]
wine: 0.6962094902992249
wine: 0.7300636172294617
wine: 0.7240580916404724
[0.696209490

In [18]:
metric='SW(HMC,F)'
for dataset in datasets:
    method='FuNNeVI'

    setup_ = get_setup(dataset)
    setup=setup_.Setup(device) 
    G=BigGenerator(lat_dim, setup.param_count, device).to(device)
    
    def projection(t,s,m):
        return setup.projection(t,s,m,ratio_ood=1)
    KLs=[]
    for i,m in models[dataset][method].items():
        t=models[dataset]['HMC'][0].to(device)
        G.load_state_dict(m)
        s=G(t.shape[0]).detach()
        K=FunSW(t,s, projection, device)
        print(dataset+': '+str(K.item()))
        KLs.append(K.item())
    print(KLs)
    results_f[metric][dataset]=(np.mean(KLs).round(decimals=3), np.std(KLs).round(decimals=3))



foong: 0.10624295473098755
foong: 0.09937095642089844
foong: 0.17299789190292358
[0.10624295473098755, 0.09937095642089844, 0.17299789190292358]
foong_sparse: 0.0741514191031456
foong_sparse: 0.02766713686287403
foong_sparse: 0.03825448825955391
[0.0741514191031456, 0.02766713686287403, 0.03825448825955391]
boston: 1.5854333639144897
boston: 1.5978670120239258
boston: 1.5927575826644897
[1.5854333639144897, 1.5978670120239258, 1.5927575826644897]
concrete: 1.000449776649475
concrete: 1.021226167678833
concrete: 0.9604642987251282
[1.000449776649475, 1.021226167678833, 0.9604642987251282]
energy: 0.6491773724555969
energy: 0.6566458940505981
energy: 0.6464962959289551
[0.6491773724555969, 0.6566458940505981, 0.6464962959289551]
powerplant: 0.6771553754806519
powerplant: 0.7039773464202881
powerplant: 0.7095881700515747
[0.6771553754806519, 0.7039773464202881, 0.7095881700515747]
wine: 0.8273176550865173
wine: 1.1735936403274536
wine: 1.8863106966018677
[0.8273176550865173, 1.17359364032

In [19]:
metric='KL(G,G)'
for dataset in datasets:
    method='GeNNeVI'

    setup_ = get_setup(dataset)
    setup=setup_.Setup(device) 
    G=BigGenerator(lat_dim, setup.param_count, device).to(device)
    models_pairs=list(itertools.combinations(models[dataset][method].items(),2))

    def projection(t,s,m):
        return setup.projection(t,s,m,ratio_ood=1)
    
    KLs=[]
    for (i,m),(j,n) in models_pairs:
        G.load_state_dict(m)
        s=G(10000).detach()
        G.load_state_dict(n)
        t=G(10000).detach()
        K=FunKL(t,s, projection,device)
        K_=FunKL(s,t, projection,device)
        print(dataset+': '+str((K.item(), K_.item())))
        KLs.append(K.item())
        KLs.append(K_.item())
    print(KLs)
    results_f[metric][dataset]=(np.mean(KLs).round(decimals=3), np.std(KLs).round(decimals=3))


foong: (88.45291137695312, 96.39619445800781)
foong: (126.31755828857422, 114.80014038085938)
foong: (174.64297485351562, 165.48837280273438)
[88.45291137695312, 96.39619445800781, 126.31755828857422, 114.80014038085938, 174.64297485351562, 165.48837280273438]
foong_sparse: (57.343963623046875, 40.75761413574219)
foong_sparse: (64.61349487304688, 33.7731819152832)
foong_sparse: (55.703617095947266, 42.70891189575195)
[57.343963623046875, 40.75761413574219, 64.61349487304688, 33.7731819152832, 55.703617095947266, 42.70891189575195]
boston: (12.182867050170898, 8.866113662719727)
boston: (9.989948272705078, 11.09997272491455)
boston: (9.40751838684082, 13.478903770446777)
[12.182867050170898, 8.866113662719727, 9.989948272705078, 11.09997272491455, 9.40751838684082, 13.478903770446777]
concrete: (5.624536037445068, 6.312488555908203)
concrete: (4.952553749084473, 7.362057685852051)
concrete: (3.038031816482544, 5.4688239097595215)
[5.624536037445068, 6.312488555908203, 4.952553749084473,

In [20]:
metric='KL(F,F)'
for dataset in datasets:
    method='FuNNeVI'

    setup_ = get_setup(dataset)
    setup=setup_.Setup(device) 
    G=BigGenerator(lat_dim, setup.param_count, device).to(device)
    models_pairs=list(itertools.combinations(models[dataset][method].items(),2))
    
    def projection(t,s,m):
        return setup.projection(t,s,m,ratio_ood=1)
    
    
    KLs=[]
    for (i,m),(j,n) in models_pairs:
        G.load_state_dict(m)
        s=G(10000).detach()
        G.load_state_dict(n)
        t=G(10000).detach()
        K=FunKL(t,s, projection,device)
        K_=FunKL(s,t, projection,device)
        print(dataset+': '+str((K.item(), K_.item())))
        KLs.append(K.item())
        KLs.append(K_.item())
    print(KLs)
    results_f[metric][dataset]=(np.mean(KLs).round(decimals=3), np.std(KLs).round(decimals=3))


foong: (52.520957946777344, 47.906219482421875)
foong: (50.27970886230469, 63.9162483215332)
foong: (52.22589874267578, 87.89900207519531)
[52.520957946777344, 47.906219482421875, 50.27970886230469, 63.9162483215332, 52.22589874267578, 87.89900207519531]
foong_sparse: (45.041561126708984, 63.74347686767578)
foong_sparse: (33.678653717041016, 56.913238525390625)
foong_sparse: (40.877010345458984, 33.108577728271484)
[45.041561126708984, 63.74347686767578, 33.678653717041016, 56.913238525390625, 40.877010345458984, 33.108577728271484]
boston: (7.901036262512207, 9.833796501159668)
boston: (10.763526916503906, 8.94189453125)
boston: (11.09880256652832, 7.776776313781738)
[7.901036262512207, 9.833796501159668, 10.763526916503906, 8.94189453125, 11.09880256652832, 7.776776313781738]
concrete: (8.639002799987793, 11.265558242797852)
concrete: (10.211370468139648, 10.495718002319336)
concrete: (10.87208366394043, 8.537967681884766)
[8.639002799987793, 11.265558242797852, 10.211370468139648, 1

In [23]:
metric='SW(G,G)'
for dataset in datasets:
    method='GeNNeVI'

    setup_ = get_setup(dataset)
    setup=setup_.Setup(device) 
    G=BigGenerator(lat_dim, setup.param_count, device).to(device)
    models_pairs=list(itertools.combinations(models[dataset][method].items(),2))

    SWs=[]
    for (i,m),(j,n) in models_pairs:
        G.load_state_dict(m)
        s=G(10000).detach()
        G.load_state_dict(n)
        t=G(10000).detach()
        SW=sw(s.cpu(),t.cpu(),'cpu')
        print(SW)
        SWs.append(SW.item())
    
    results_f[metric][dataset]=(np.mean(SWs).round(decimals=3), np.std(SWs).round(decimals=3))


0.9833998963262862
1.0010130740949448
1.0596030567016705
0.8829152594599982
0.8620348148315402
0.7587924829108503
0.3170504052930995
0.3342826169958495
0.3146595967053306
0.3033324652869945
0.29069816075734284
0.3200016773846258
0.41550359281600224
0.41131986530968917
0.37912868121005966
0.28505819939600635
0.28208509802233606
0.29092822727596357
0.837964136209949
0.9071994995901592
0.8388422843920721
0.3171396785918505
0.3142767687658171
0.31381316762689043


In [24]:
metric='SW(F,F)'
for dataset in datasets:
    method='FuNNeVI'

    setup_ = get_setup(dataset)
    setup=setup_.Setup(device) 
    G=BigGenerator(lat_dim, setup.param_count, device).to(device)
    models_pairs=list(itertools.combinations(models[dataset][method].items(),2))

    SWs=[]
    for (i,m),(j,n) in models_pairs:
        G.load_state_dict(m)
        s=G(10000).detach()
        G.load_state_dict(n)
        t=G(10000).detach()
        SW=sw(s.cpu(),t.cpu(),'cpu')
        print(SW)
        SWs.append(SW.item())
    
    results_f[metric][dataset]=(np.mean(SWs).round(decimals=3), np.std(SWs).round(decimals=3))


1.214326170592011
2.567463420543314
2.801676671520815
2.332224825621222
2.385387473239601
1.6965247560980528
1.8402081713275193
1.8494151857217
1.8681053212998984
2.1406877663195663
1.8864331752861316
1.9064292389639022
1.645641865504179
1.6122059645543578
1.529769052856103
2.016032211031002
1.8632366358260748
2.2603834422468494
0.7904368752396598
0.821818331954727
0.8575065940734043
2.508695002268155
2.2402337711643696
2.379821249066939


In [25]:
torch.save(results_f, 'Results/results_f_es.pt')
pd.DataFrame(results_f)


,"KL(G,HMC)","KL(F,HMC)","KL(HMC,G)","KL(HMC,F)","SW(HMC,G)","SW(HMC,F)","KL(G,G)","KL(F,F)","SW(G,G)","SW(F,F)"
foong,"(117.032, 29.862)","(80.435, 4.338)","(104.637, 16.706)","(74.029, 27.492)","(0.068, 0.022)","(0.126, 0.033)","(127.683, 32.456)","(59.125, 13.822)","(1.015, 0.033)","(2.194, 0.7)"
foong_sparse,"(63.782, 9.078)","(81.117, 7.483)","(40.699, 6.101)","(55.61, 4.739)","(0.017, 0.003)","(0.047, 0.02)","(49.15, 10.783)","(45.56, 11.387)","(0.835, 0.054)","(2.138, 0.313)"
boston,"(66.57, 1.653)","(10.645, 1.134)","(9.45, 0.279)","(52.459, 1.125)","(0.425, 0.004)","(1.592, 0.005)","(10.838, 1.608)","(9.386, 1.293)","(0.322, 0.009)","(1.853, 0.012)"
concrete,"(64.879, 2.005)","(12.017, 0.788)","(6.43, 0.143)","(45.99, 1.06)","(0.315, 0.003)","(0.994, 0.025)","(5.46, 1.323)","(10.004, 1.052)","(0.305, 0.012)","(1.978, 0.115)"
energy,"(41.331, 5.411)","(13.591, 0.558)","(15.817, 10.178)","(48.682, 0.846)","(0.379, 0.131)","(0.651, 0.004)","(36.601, 13.169)","(10.288, 0.712)","(0.402, 0.016)","(1.596, 0.049)"
powerplant,"(89.942, 2.464)","(18.106, 0.184)","(7.952, 0.382)","(70.03, 1.162)","(0.105, 0.0)","(0.697, 0.014)","(28.022, 10.154)","(16.081, 0.742)","(0.286, 0.004)","(2.047, 0.164)"
wine,"(86.514, 2.826)","(83.666, 8.417)","(22.461, 1.213)","(35.132, 4.398)","(0.717, 0.015)","(1.296, 0.441)","(76.391, 2.215)","(83.093, 14.072)","(0.861, 0.032)","(0.823, 0.027)"
yacht,"(101.263, 1.163)","(13.847, 2.354)","(14.17, 0.525)","(51.78, 1.576)","(0.244, 0.007)","(0.608, 0.004)","(26.103, 10.869)","(9.466, 2.852)","(0.315, 0.001)","(2.376, 0.11)"
